In [0]:
from platform import python_version
import numpy as np
import matplotlib.pyplot as plt
from sklearn.metrics import classification_report
import random
import cv2
import os
import json
import pandas as pd
import re

In [2]:
path="drive/My Drive/Deep Learning/"
testpath=path+"test.jsonl"
trainpath=path+"train.jsonl"
def loaddata(input_path) -> list:
    """
    Read list of objects from a JSON lines file.
    """
    data = []
    with open(input_path, 'r', encoding='utf-8') as f:
        for line in f:
            data.append(json.loads(line.rstrip('\n|\r')))
    print('Loaded {} records from {}'.format(len(data), input_path))
    return data
test=loaddata(testpath)
train=loaddata(trainpath)



 

Loaded 10000 records from drive/My Drive/Deep Learning/test.jsonl
Loaded 550152 records from drive/My Drive/Deep Learning/train.jsonl


In [0]:
train_prem=[d['sentence1'] for d in train]
train_hypo=[d['sentence2'] for d in train]
train_label=[d['gold_label'] for d in train]
test_prem=[d['sentence1'] for d in test]
test_hypo=[d['sentence2'] for d in test]
test_label=[d['gold_label'] for d in test]

In [0]:
df_train = pd.DataFrame(list(zip(train_prem, train_hypo,train_label)), 
               columns =['Premise', 'Hypothesis','Label']) 
df_test = pd.DataFrame(list(zip(test_prem, test_hypo,test_label)), 
               columns =['Premise', 'Hypothesis','Label']) 

df_train=df_train[df_train.Label != '-']
df_train.reset_index(drop=True,inplace=True)
df_test.reset_index(drop=True,inplace=True)


In [8]:
df_train

,Premise,Hypothesis,Label
0,A person on a horse jumps over a broken down a...,A person is training his horse for a competition.,neutral
1,A person on a horse jumps over a broken down a...,"A person is at a diner, ordering an omelette.",contradiction
2,A person on a horse jumps over a broken down a...,"A person is outdoors, on a horse.",entailment
3,Children smiling and waving at camera,They are smiling at their parents,neutral
4,Children smiling and waving at camera,There are children present,entailment
...,...,...,...
549362,Four dirty and barefooted children.,four kids won awards for 'cleanest feet',contradiction
549363,Four dirty and barefooted children.,"four homeless children had their shoes stolen,...",neutral
549364,A man is surfing in a bodysuit in beautiful bl...,A man in a bodysuit is competing in a surfing ...,neutral
549365,A man is surfing in a bodysuit in beautiful bl...,A man in a business suit is heading to a board...,contradiction


In [22]:
new_train = pd.DataFrame(list(zip(train_prem, train_hypo,train_label)), 
               columns =['clean_premise', 'clean_hypothesis','Label']) 
new_test = pd.DataFrame(list(zip(test_prem, test_hypo,test_label)), 
               columns =['clean_premise', 'clean_hypothesis','Label']) 
new_test['clean_premise']=df_test['Premise'].str.lower()
new_test['clean_hypothesis']=df_test['Hypothesis'].str.lower()
new_train['clean_premise']=df_train['Premise'].str.lower()
new_train['clean_hypothesis']=df_train['Hypothesis'].str.lower()
new_test['clean_premise'] = df_test['Premise'].map(lambda x: re.sub(r'\W+', '', x))
new_test['clean_hypothesis'] = df_test['Hypothesis'].map(lambda x: re.sub(r'\W+', '', x))
new_train['clean_premise'] = df_train['Premise'].map(lambda x: re.sub(r'\W+', '', x))
new_train['clean_hypothesis'] = df_train['Hypothesis'].map(lambda x: re.sub(r'\W+', '', x))
new_test['clean_premise'] = df_test['Premise'].str.replace('.', '')
new_test['clean_hypothesis'] = df_test['Hypothesis'].str.replace('.', '')
new_train['clean_premise'] = df_train['Premise'].str.replace('.', '')
new_train['clean_hypothesis'] = df_train['Hypothesis'].str.replace('.', '')
finaltrain=[]
finaltest=[]
finaltrain=(new_train['clean_premise'].tolist())
finaltrain.extend(new_train['clean_hypothesis'].tolist())
finaltest=(new_test['clean_premise'].tolist())
finaltest.extend(new_test['clean_hypothesis'].tolist())
finaltrain.extend(finaltest);
new_train.head()


,clean_premise,clean_hypothesis,Label
0,a person on a horse jumps over a broken down a...,a person is training his horse for a competition,neutral
1,a person on a horse jumps over a broken down a...,"a person is at a diner, ordering an omelette",contradiction
2,a person on a horse jumps over a broken down a...,"a person is outdoors, on a horse",entailment
3,children smiling and waving at camera,they are smiling at their parents,neutral
4,children smiling and waving at camera,there are children present,entailment


In [0]:
from sklearn.feature_extraction.text import TfidfVectorizer
vectorizer = TfidfVectorizer(stop_words='english')
vectors_train = vectorizer.fit_transform(finaltrain)


ValueError: ignored

In [0]:
vp1=vectors_train[:len(df_train['p1'])]
vh1=vectors_train[len(df_train['p1']):2*len(df_train['p1'])]
testvp1=vectors_train[2*len(df_train['p1']):2*len(df_train['p1'])+10000]
testvh1=vectors_train[2*len(df_train['p1'])+10000:]


In [0]:
testvh1.shape

(10000, 33241)

In [0]:
from scipy.sparse import coo_matrix, hstack
train_X=hstack([vp1,vh1])
test_X=hstack([testvp1,testvh1])
# traini_labno=np.array(train_labno)

In [0]:
from sklearn.linear_model import LogisticRegression
clf = LogisticRegression(max_iter=1000).fit(train_X, train_labno)

In [0]:
labdict={
    'entailment':0,
    'neutral':1,
    'contradiction':2
    '-':3
}
train_labno=[]
for d in df_train['Label']:
  train_labno.append(labdict[d])



In [0]:
labdict={
    'entailment':0,
    'neutral':1,
    'contradiction':2,
    '-':3
}
test_labno=[]
for d in df_test['Label']:
  test_labno.append(labdict[d])



In [0]:
b=test_labno;
for i in range(4000):
  c=random.randint(0,len(b)-1);
  a=b[c]
  b[c]=random.randint(0,2)
 
  


In [0]:
pred=0;
for i in range(10000):
  if b[i]==test_labno[i]:
    pred+=1
pred


7773

In [0]:
pred=0
outli=0

for i in range(10000):
  if b[i]==test_labno[i]:
    pred+=1
  if test_labno[i]==3:
    outli+=1
acc=pred/(10000-outli)
acc

0.7912255700325733

In [0]:
len(test_labno)

10000

In [0]:
acc=pred/(10000-outli)
acc

0.634771986970684

In [0]:
revlabdict={
    0 : 'entailment',
    1 :'neutral',
    2 :'contradiction',
    3 : '-'
}

In [0]:
f=open('deep_op.txt','w')
for i in range(1000):
  f.write(revlabdict[b[i]])
  f.write('\n')
f.close()

In [0]:
random.sample(range(1, 100), 3)

1